In [44]:
using ITensors
using Random
include("real_complex_final_fixed.jl");

In [13]:
dset = rand(10, 10)
labels = rand([0, 1], 10);
s = siteinds("S=1/2", 10);
mps = randomMPS(ComplexF64, s; linkdims=5);

In [14]:
training_pstates = dataset_to_product_state(dset, labels, s);

manual check inner product/overlap

In [16]:
ps = training_pstates[1].pstate

MPS
[1] ((dim=2|id=564|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=783|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=315|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=327|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=380|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=527|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=193|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=977|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=51|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=270|"S=1/2,Site,n=10"),)


In [39]:
inner(ps, mps)

-0.025238215706567674 - 0.01038481154450671im

In [40]:
LE, RE = construct_caches(mps, training_pstates; going_left=false)

(ITensor[#undef #undef … #undef #undef; #undef #undef … #undef #undef; … ; #undef #undef … #undef #undef; #undef #undef … #undef #undef], ITensor[ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
-0.025238215706567677 - 0.0103848115445067im ITensor ord=1
Dim 1: (dim=5|id=752|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 5-element
 -0.021516117530988345 + 0.026287717820871706im
 -0.001041757488820583 + 0.007372456495327922im
 -0.021497167287010864 - 0.016484188391739815im
 -0.005935328386858356 - 0.036177915424446504im
 -0.014658854781395425 - 0.00502561183358491im … ITensor ord=1
Dim 1: (dim=4|id=312|"Link,l=8")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4-element
   0.5359304965014184 - 0.49287587096434826im
 -0.04117689697622809 + 0.2579355491937334im
 -0.27001897743763453 + 0.04471106123561237im
  -0.5026463779417771 + 0.27214497226137896im ITensor ord=1
Dim 1: (dim=2|id=146|"Link,l=9")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 

In [32]:
BT = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=564|"S=1/2,Site,n=1") (dim=2|id=783|"S=1/2,Site,n=2") (dim=5|id=397|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [43]:
@show BT * RE[1, 3] * conj(ps[1]) * conj(ps[2])

BT * RE[1, 3] * conj(ps[1]) * conj(ps[2]) = ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
-0.02523821570656768 - 0.010384811544506706im


ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [33]:
C_index = Index(2, "C")
bt_real = real(BT)
bt_imag = imag(BT)

ITensor ord=3 (dim=2|id=564|"S=1/2,Site,n=1") (dim=2|id=783|"S=1/2,Site,n=2") (dim=5|id=397|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [34]:
bt_real_index_tensor = ITensor([1; 0], C_index)
bt_real *= bt_real_index_tensor
bt_imag_index_tensor = ITensor([0; 1], C_index)
bt_imag *= bt_imag_index_tensor

ITensor ord=4 (dim=2|id=564|"S=1/2,Site,n=1") (dim=2|id=783|"S=1/2,Site,n=2") (dim=5|id=397|"Link,l=2") (dim=2|id=780|"C")
NDTensors.Dense{Float64, Vector{Float64}}

In [35]:
bt_combined_real_imag = bt_real + bt_imag

ITensor ord=4 (dim=2|id=564|"S=1/2,Site,n=1") (dim=2|id=783|"S=1/2,Site,n=2") (dim=5|id=397|"Link,l=2") (dim=2|id=780|"C")
NDTensors.Dense{Float64, Vector{Float64}}

In [45]:
yhat, d_yhat_dW = compute_yhat_and_derivative(bt_combined_real_imag, LE, RE, training_pstates[1], 1, 2)

(ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
-0.02523821570656768 - 0.010384811544506708im, ITensor ord=3
Dim 1: (dim=2|id=564|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=783|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=397|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
  0.018923847274471765 + 0.006800728343477763im  …    0.0327766089007112 + 0.013601884883983322im
 -0.004409836698904342 + 0.005092374934531295im     -0.00820756744260228 + 0.0086000549807179im

[:, :, 2] =
 0.005289672508348033 - 0.011435863538912534im   …  0.010299588318087126 - 0.0197065642581231im
 0.002934916997445232 + 0.0030336174440655007im     0.004914540419911635 + 0.005597679995856064im

[:, :, 3] =
 -0.0018980865229370153 - 0.017155889668963427im   …  -0.0018821975082217726 - 0.030402334056147572im
   0.005590133900866669 + 0.0014781308030236988im       0.009727543812326161 + 0.003082358955740567im

[:, :, 4] =
 -0.013567425111544209 - 0.00189261369518474im   …  

In [46]:
compute_loss_and_gradient_per_sample(bt_combined_real_imag, LE, RE, training_pstates[1], 1, 2)

2-element Vector{Any}:
 0.5256106216280007
  ITensor ord=3
Dim 1: (dim=2|id=564|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=783|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=397|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
 -0.019472075696195426 + 0.006775846004729807im  …  -0.03374508503744739 + 0.013604793282199308im
   0.00446824975473295 + 0.005266692714647884im     0.008325401849897726 + 0.008902339064539853im

[:, :, 2] =
 -0.0053044151164305196 - 0.011779416581829832im  …  -0.010354882593737534 - 0.020310882061374283im
 -0.0030404926111623747 + 0.003079701975572794im     -0.005096705502971431 + 0.005687918874959175im

[:, :, 3] =
 0.002124151521123163 - 0.01756916244223093im    …  0.0022454233246228684 - 0.031150088454627328im
 -0.00574656901591287 + 0.0014573837000038743im     -0.010005059378224624 + 0.003059133486668678im

[:, :, 4] =
   0.01392949714964311 - 0.001799484734945422im  …   0.024399792242530915 - 0.004360216198419926im
 -0.002244042799788729 - 0.0041